In [189]:
#default_exp blocks

Esse código é baseado no `driver.py` do Arthur: https://git.anatel.gov.br/anatel/rfeye
O código contempla as mesmas classes mas com implementação bastante distinta:
* Eliminação de métodos que somente retornam atributos para atributos diretamente.
* Eliminação de Repetição de Código 
* Otimização de Loops para List Comprehensions
* Mudança de Herança para Composição e delegação de atributos da atual para a antiga superclasse, agora como atributo

In [190]:
#export
from datetime import datetime
from typing import Union, Any, Optional, List, Tuple, Iterator, Mapping, Set
from rfpy.utils import *
from fastcore.foundation import *
path_type = Union[str, Any]
bin_val = Union[int, bytes]
bytes_encoded = Union[int, bytes]
datetime_object = datetime

### BinFileStats

In [191]:
#export
# Estatísticas do arquivo e dos blocos
class BinFileStats:
    """
    Esta classe Fornece alguns dados de resumo sobre o arquivo bin e uma visão geral dos blocos contidos nele.
    Pode ser usado para criar uma base de dados de informações dos arquivos sem estruturar os dados propriamente ditos,
    ou para fazer pré-processamentos e filtros rápidos.

    Um arquivo sempre possui:
        > um bloco de cabecalho,
        > varios blocos de dados no corpo e
        > um bloco de rodape que é opcional.
    """
    timed_blocs_types = (63, 65)  # blocos com marca temporal e NDATA > 0

    def __init__(self, file_name: path_type) -> None:
        self.file_name = file_name
        # instancia um gerador que fornece um bloco por vez a partir do arquivo de entrada.
        self._file_gen = binary_file_generator(file_name)
        # extrai as informações do cabeçalho.
        self._file_header()
        # passa pelo arquivo montando as estatisticas.
        self.footer, raw_rodape = self._file_stats()
        # extrai as informações do rodapé - se existir.
        if self.footer:
            self._file_footer(raw_rodape)

    def _file_header(self) -> None:
        """
        Constroi o cabeçalho do arquivo.
        :return: None
        """
        # O primeiro bloco do arquivo é o cabeçalho e tem 36 bytes de tamanho.
        bin_raw = next(self._file_gen, None)
        self._file_version_num = bin2dec(bin_raw[:4])
        self._file_version_text = bin2str(bin_raw[4:])

    def _file_stats(self) -> Tuple[bool, Optional[bytes]]:
        """
        self > bool, bytes
        :return: True, bytes se existir rodapé no arquivo. False, None se não existir.

        Constroi uma estatística dos blocos do arquivo.
        """
        # inicializa as variaveis adequadamente
        self.mbr_latitude_bottom: float = 90.0
        self.mbr_latitude_top: float = -90.0
        self.mbr_longitude_left: float = 180.0
        self.mbr_longitude_right: float = -180.0
        self.wall_clock_first_start_date_time: datetime_object = datetime.now()
        self.wall_clock_last_stop_date_time: datetime_object = datetime(1900, 1, 1, 00, 00, 00, 00)
        self.start_mega: int = 0
        self.stop_mega: int = 0
        self.type_cont: Mapping[int, int] = {}
        self.threadid_cont: Mapping[int, int] = {}
        self.threadid_ndata: Mapping[int, Set[int]] = {}
        self.threadid_band: Mapping[int, Set[Tuple[int, int]]] = {}
        old_block = None
        # circula pelo arquivo
        for bin_block in self._file_gen:
            block = Block(bin_block)
            old_block = block
            blk_tid = block.thread_id
            blk_type = block.block_type
            # mapeia as ocorrências de cada thread ID
            self.threadid_cont[blk_tid] = self.threadid_cont.get(blk_tid, 0) + 1
            if blk_type in self.timed_blocs_types:
                Classe = retorna_classe(blk_type)
                c_block = Classe(bin_block)
                # mapeia a quantidade de "steps" de cada thread ID.
                newset = self.threadid_ndata.get(blk_tid, {c_block.data_points})
                newset.add(c_block.data_points)
                self.threadid_ndata[blk_tid] = newset
                # mapeia as faixas de frequencias de cada thread ID.
                newset = self.threadid_band.get(blk_tid, {c_block.freq_band_mega})
                newset.add(c_block.freq_band_mega)
                self.threadid_band[blk_tid] = newset
            # mapeia as ocorrências de cada tipo de bloco
            self.type_cont[blk_type] = self.type_cont.get(blk_type, 0) + 1
            # informações geográficas
            if blk_type == 40:  # dados de GPS
                gps_block = DataType40(bin_block)
                latitude = gps_block.latitude
                if latitude < self.mbr_latitude_bottom:
                    self.mbr_latitude_bottom = latitude
                if latitude > self.mbr_latitude_top:
                    self.mbr_latitude_top = latitude
                longitude = gps_block.longitude
                if longitude < self.mbr_longitude_left:
                    self.mbr_longitude_left = longitude
                if longitude > self.mbr_longitude_right:
                    self.mbr_longitude_right = longitude
            # informações temporais e de frequencias
            if blk_type in (63, 65):  # Spectral Data or Frequency Channel Occupancy
                tblock = TimedBlock(bin_block)
                date_time_stamp = tblock.datetime_stamp
                if date_time_stamp < self.wall_clock_first_start_date_time:
                    self.wall_clock_first_start_date_time = date_time_stamp
                if date_time_stamp > self.wall_clock_last_stop_date_time:
                    self.wall_clock_last_stop_date_time = date_time_stamp
                start_mega = tblock.start_mega
                if self.start_mega == 0 or self.start_mega > start_mega:
                    self.start_mega = start_mega
                stop_mega = tblock.stop_mega
                if self.stop_mega < self.start_mega or self.stop_mega < stop_mega:
                    self.stop_mega = stop_mega
            # informações sobre a estação e o LOGGER
            if blk_type == 21:  # Unit and Job Information
                ui = DataType21(bin_block)
                self.hostname = ui.hostname
                self.unit_info = ui.unit_info
                self.method = ui.method


        if old_block.block_type == 20:  # rodapé
            return True, old_block
        else:
            return False, None

    def _file_footer(self, binary_raw_footer) -> None:
        """
        Constroi o rodapé do arquivo - se existir.
        :return: None

        Retorna os varios campos do bloco de rodape do arquivo.
        Considera que o bloco 'binary_raw_footer' é o rodape inteiro.

        Tamanho do rodape (bytes) = 40 + (8 * NDDT)

        An optional data block at the end of each file provides a summary of some aspects of the file content.
        The MBR is the Minimum Bounding Rectangle of all GPS records that have a fix.
        The start and end date/time is included, omitting the nanoseconds fields.
        If a file is used for the output from more than one unit, multiple footers may appear, up to one per unit.
        todo: implementar uma rotina de extração de rodapé compatível com múltiplos blocos de rodapé.

        Field Bytes Description                             Comments
            0   4   MBR: bottom latitude                    Minimum Bounding Rectangle of all GPS with fix, ..
            1   4   MBR: left longitude                     .. in units of integer microdegrees (same as ..
            2   4   MBR: top latitude                       .. GPS blocks)
            3   4   MBR: right longitude                    “
            4   4   Start Date (Wall Clock)                 Date of first data block (dd/mm/yy/null), see section 4
            5   4   Start Time (Wall Clock)                 Time of first data block (hh/mm/ss/cc), see section 4
            6   4   End Date (Wall Clock)                   Date of last data block (dd/mm/yy/null), see section 4
            7   4   End Time (Wall Clock)                   Time of last data block (hh/mm/ss/cc), see section 4
            8   4   Total no. of spectral data blocks       Total number of blocks in threads with threadID>0
            9   4   NDDT = Number of distinct data threads  Spectral data, i.e. ThreadID > 0.
                                                            Optionally can be set to a special value of zero,
                                                            to omit the following information.
            // Start of [Repeated * NDDT]
            10  4   ThreadID                                Fields (10,11) are repeated (Field 9) times.
            11  4   Number of blocks
            // End of [Repeated * NDDT]
        """

        def help_ddt_generator(raw_ddt_seq: bytes) -> Tuple[int, int]:
            """
            bytes > int, int
            :param raw_ddt_seq: Binario contendo as estatisticas de threadIds
            :return: O número do ThreadID e a contagem de blocos com este número.
            """
            field_length = 4
            start_pos = 0
            while start_pos <= len(raw_ddt_seq):
                thread_id = raw_ddt_seq[start_pos:field_length]
                num_blocks = raw_ddt_seq[start_pos+field_length:field_length]
                start_pos += field_length*2
                yield bin2dec(thread_id), bin2dec(num_blocks)

        # checa se o campo é realmente um rodape
        if not bin2dec(binary_raw_footer[:4]) == 20:
            return None
        # sendo rodapé, destrincha as informações:
        # coordenadas geográficas
        self.footer_mbr_latitude_bottom = bin2dec(binary_raw_footer[:4])
        self.footer_mbr_longitude_left = bin2dec(binary_raw_footer[4:8])
        self.footer_mbr_latitude_top = bin2dec(binary_raw_footer[8:12])
        self.footer_mbr_longitude_right = bin2dec(binary_raw_footer[12:16])
        # tempos e datas
        self.footer_wall_clock_start_date = bin2date(binary_raw_footer[16:20])
        self.footer_wall_clock_start_time = bin2time(binary_raw_footer[20:24])
        self.footer_wall_clock_stop_date = bin2date(binary_raw_footer[24:28])
        self.footer_wall_clock_stop_time = bin2time(binary_raw_footer[28:32])
        # outras informações sobre blocos
        self.footer_spectral_data_blocks = bin2dec(binary_raw_footer[32:36])
        self.footer_distinct_threadids = bin2dec(binary_raw_footer[32:36])
        raw_ddt = binary_raw_footer[36:]
        self.footer_threadid_cont = {}  # A chave é o número da ThreadID e o valor a quantidade de blocos
        for tid, cont in help_ddt_generator(raw_ddt):
            self.footer_threadid_cont[tid] = cont

    def _get_file_version_num(self) -> int:
        """
        self > int
        :return: A versão do arquivo bin.
        """
        return self._file_version_num

    def _get_file_version_text(self) -> str:
        """
        self > str
        :return: A descrição do arquivo bin.
        """
        return self._file_version_text

    def _get_mbr(self) -> Tuple[float, float, float, float]:
        """

        :return: O quadrilátero gerográfico que contém todas as medidas.
        """
        return self.mbr_latitude_top, self.mbr_longitude_left, self.mbr_latitude_bottom, self.mbr_longitude_right

    def _get_mbr_latitudes(self) -> Tuple[float, float]:
        """

        :return: As latitudes máxima e mínima.
        """
        return \
            max(self.mbr_latitude_top, self.mbr_latitude_bottom), \
            min(self.mbr_latitude_top, self.mbr_latitude_bottom)

    def _get_mbr_longitudes(self) -> Tuple[float, float]:
        """

        :return: As longitudes mínima e máxima.
        """
        return \
            min(self.mbr_longitude_left, self.mbr_longitude_right), \
            max(self.mbr_longitude_left, self.mbr_longitude_right)

    def _get_time_interval(self) -> Tuple[datetime_object, datetime_object]:
        """

        :return: As marcas temporais da primeira e da última medida.
        """
        return self.wall_clock_first_start_date_time, self.wall_clock_last_stop_date_time

    def _get_frequency_band(self) -> Tuple[int, int]:
        """

        :return: As frequencias mais baixa e mais alta em MHz.
        """
        return self.start_mega, self.stop_mega

    def _get_threadids(self) -> Mapping[int, int]:
        """

        :return: um mapa da quantidade de blocos por threadIDs
        """
        return self.threadid_cont

    def _get_threadids_ndata(self) -> Mapping[int, Set[int]]:
        """

        :return: um mapa da quantidade de blocos por threadIDs
        """
        return self.threadid_ndata

    def _get_threadids_band(self) -> Mapping[int, Set[Tuple[int, int]]]:
        """

        :return: um mapa da quantidade de blocos por threadIDs
        """
        return self.threadid_band

    def _get_types(self) -> Mapping[int, int]:
        """

        :return: um mapa da quantidade de blocos por threadIDs
        """
        return self.type_cont

    def __len__(self) -> int:
        """
        self > int
        :return: A contagem de blocos
        """
        total = 0
        for num in self.type_cont:
            total += self.type_cont[num]
        return total

    def __str__(self) -> str:
        """
        self > str
        :return: A descrição do tipo e do tamanho do bloco em bytes
        """
        return f"Objeto '{self._get_file_version_text()}' contendo {self.__len__()} blocos: '{self.file_name}'."

    file_version_num: int = property(fget=_get_file_version_num)
    file_version_text: int = property(fget=_get_file_version_text)
    mbr: Tuple[float, float, float, float] = property(fget=_get_mbr)
    mbr_latitudes: Tuple[float, float] = property(fget=_get_mbr_latitudes)
    mbr_longitudes: Tuple[float, float] = property(fget=_get_mbr_longitudes)
    time_interval: Tuple[datetime_object, datetime_object] = property(fget=_get_time_interval)
    frequency_band: Tuple[int, int] = property(fget=_get_frequency_band)
    thread_ids_count: Mapping[int, int] = property(fget=_get_threadids)
    thread_ids_frequency_band: Mapping[int, int] = property(fget=_get_threadids_band)
    thread_ids_ndata: Mapping[int, Set[int]] = property(fget=_get_threadids_ndata)
    types_count: Mapping[int, int] = property(fget=_get_types)

### Bloco Genérico

In [192]:
#export
class Block:
    """
    Um bloco é um pedaço do arquivo .bin com inicio e final conhecidos e que contém diferentes tipos de informação.
    Possui varios campos: tipo, cabeçalho, dados e rodape.
    Cada campo possui comprimentos e informações definidas na documentação.
    """
    def __init__(self, binary_data: bytes) -> None:
        """
        bytes > None
        :param binary_data: Um único bloco no formato de uma sequencia binária válida.
        """
        self._raw_bin = self.assert_raw_bin(binary_data)
        self._block_header = self._get_block_header()
        self._block_trailer = self._get_block_trailer()
        #self.block_data = self._get_block_data() #Os bloco sem cabeçalho e sem maiores tratamentos.


    @staticmethod
    def assert_raw_bin(bin_block):
        data_type = type(bin_block)
        if data_type is not bytes:
            print(f"Expected type 'bytes', got '{data_type}' instead.")
            raise AssertionError(f"Expected type 'bytes'.")
        return bin_block

    def _get_block_header(self) -> Tuple[Union[int, bytes], Union[int, bytes], Union[int, bytes]]:
        """
        self > tuple(int, int, int)
        :return: Thread Id, tamanho em bytes, tipo do bloco

        O cabecalho de um bloco possui 12 bytes de comprimento
        F0 = (4 bytes) threadID
        F1 = (4 bytes) Block Size
        F2 = (4 bytes) Data Type
        """
        thread_id = self._raw_bin[:4]
        block_size = self._raw_bin[4:8]
        block_type = self._raw_bin[8:12]
        return thread_id, block_size, block_type

    def _get_thread_id(self) -> int:
        """
        self > int
        :return: O threadID do bloco.
        """
        return bin2dec(self._block_header[0])

    def _get_block_size(self) -> int:
        """
        self > int
        :return: O tamanho do bloco em bytes.
        """
        return bin2dec(self._block_header[1])

    def _get_block_type(self) -> int:
        """
        self > int
        :return: O tipo do bloco.
        """
        return bin2dec(self._block_header[2])

    def _get_block_data(self) -> bytes:
        """
        self > bytes
        :return: Os bloco sem cabeçalho e sem maiores tratamentos.
        """
        return self._raw_bin[12:]

    def _get_block_trailer(self) -> bytes:
        """
        self > bytes
        :return: O rodapé do bloco

        O rodapé possui 8 bytes de comprimento.
        F0 = (4 bytes) checksum
        F1 = (4 bytes) EOF marker que é sempre 0x55555555
        """
        return self._raw_bin[-4:]

    def _checksum(self) -> bool:
        """
        self > bool
        :return: True se o calculo do checksum estiver correto ou False se o calculo do checksum estiver incorreto.

        todo: pedir ajuda no cálculo:
            aqui eu não sei somar todos os bytes do bloco
            ou pegar os 32bits menos significativos para comparar com o checksum
        """
        pass
        # testar:
#       sumbyte = b'\x00'
#       for databyte in self._raw_bin[:-8]:
#           sumbyte +=  databyte

    def __str__(self) -> str:
        """
        self > str
        :return: A descrição do tipo e do tamanho do bloco em bytes
        """
        return f"Bloco do tipo {self._get_block_type()} com {self._get_block_size()} bytes de tamanho."

    block_type: int = property(fget=_get_block_type)
    thread_id: int = property(fget=_get_thread_id)
    size_bytes: int = property(fget=_get_block_size)
    block_data: bytes = property(fget=_get_block_data)

### TimedBlock

In [193]:
#export
class TimedBlock(Block):
    """
    Aplicável aos tipos de bloco:
        - DataType65
        - DataType63

    Implementa o mapeamento de funções dos blocos que possuem os seguintes campos
    (na mesma posição do vetor binario):F0 a F8

    F0 = (4 bytes) WALLDATE = Wall Clock Start Date of measurements
    F1 = (4 bytes) WALLTIME = Wall Clock Start Time
    F2 = (4u bytes) WALLNANO = Wall Clock Start Time Nanoseconds
    F3 = (2u bytes) STARTMEGA = Start Frequency MHz
    F4 = (4 bytes) STARTMILLI = Start Frequency mHz
    F5 = (2u bytes) STOPMEGA = Stop Frequency MHz
    F6 = (4 bytes) STOPMILLI = Stop Frequency mHz
    F7 = (2u bytes) STARTCHAN = Start Channel number
    F8 = (2u bytes) STOPCHAN = Stop Channel number

    Outros campos em posições diferentes do vetor binario devem possuir métodos na classe herdada:
    NAMAL, ANTUID, PROC, DTYPE, GERROR, GFLAGS, GROUPID, NPAD e NDATA

    """
    # used in PROC: self._get_processing()
    dict_processing: Mapping[int, str] = {0: 'single measurement', 1: 'average', 2: 'peak', 3: 'minimum'}
    # used in DTYPE: self._get_unit()
    dict_unit: Mapping[int, str] = {0: '%', 1: 'dBm', 2: 'dBuV/m'}

    def __init__(self, data)-> None:
        super().__init__(data)
        #self.block = block
        self.bw = self._get_stop_mega() - self._get_start_mega()
        self.passo = self.bw/(self._get_number_data_points()-1)
#        self.list_freq_mega = [self.point_freq_mega(index) for index in range(self._get_number_data_points())]

#     def __getattr__(self, k):
#         return delegate_attr(self, k, to='block')


    def _get_date_stamp(self) -> Tuple[int, int, int, int]:
        """
        self > int, int, int, int
        :return: O carimbo de data de início da medida: dia, mês, ano, reserva

        F0 = (4 bytes) WALLDATE = Wall Clock Start Date of measurements
        """
        return bin2date(self.block_data[:4])

    def _get_time_stamp_nanosecs(self) -> Tuple[int]:
        """
        self > int
        :return: Nanossegundos da hora de inicío da medida.

        F2 = (4u bytes) WALLNANO = Wall Clock Start Time Nanoseconds
        """
        return bin2dec(self.block_data[8:12], False),

    def _get_time_stamp(self) -> Tuple[int, int, int, int]:
        """
        self > int, int, int, int
        :return: Carimbo de tempo do início da medida: horas, minutos, segundos, décimos de segundo

        F1 = (4 bytes) WALLTIME = Wall Clock Start Time
        """
        return bin2time(self.block_data[4:8])

    def _get_datetime_tuples(self) -> Tuple[Tuple[int, int, int, int], Tuple[int, int, int, int], Tuple[int]]:
        """
        self > (int, int, int, int), (int, int, int, int), (int,)
        :return: Um carimbo de data e hora completo (incluindo nanossegundos) do início da medida.
        """
        return self._get_date_stamp(), self._get_time_stamp(), self._get_time_stamp_nanosecs()

    def _get_datetime_stamp(self) -> datetime:
        """
        self > datetime
        :return: Objeto datetime do início da medida.
        """
        date_stamp: Tuple[int, int, int, int] = self._get_date_stamp()
        time_stamp: Tuple[int, int, int, int] = self._get_time_stamp()
        microsecs: int = int(self._get_time_stamp_nanosecs()[0]/1000)
        return datetime(2000+date_stamp[2], date_stamp[1], date_stamp[0],
                        time_stamp[0], time_stamp[1], time_stamp[2], microsecs)

    def _get_start_mega(self) -> int:
        """
        self > int
        :return: A frequência inicial em megahertz.

        F3 = (2u bytes) STARTMEGA = Start Frequency MHz
        """
        return bin2dec(self.block_data[12:14], False)

    def _get_start_mili(self) -> int:
        """
        sellf > int
        :return: A frequência inicial em milihertz.

        F4 = (4 bytes) STARTMILLI = Start Frequency mHz
        """
        return bin2dec(self.block_data[14:18])

    def _get_stop_mega(self) -> int:
        """
        sellf > int
        :return: A frequência final em megahertz.

        F5 = (2u bytes) STOPMEGA = Stop Frequency MHz
        """
        return bin2dec(self.block_data[18:20], False)

    def _get_stop_mili(self) -> int:
        """
        sellf > int
        :return: A frequência final em milihertz

        F6 = (4 bytes) STOPMILLI = Stop Frequency mHz
        """
        return bin2dec(self.block_data[20:24])

    def _get_freq_band_mega(self) -> Tuple[int, int]:
        """
        self > (int, int)
        :return: As frequências inicial e final em megahertz.
        """
        return self._get_start_mega(), self._get_stop_mega()

    def _get_freq_band_mili(self) -> Tuple[int, int]:
        """
        self > (int, int)
        :return: As frequências inicial e final em milihertz.
        """
        return self._get_start_mili(), self._get_stop_mili()

    def _get_channel(self) -> Tuple[int, int]:
        """
        self > (int, int)
        :return: O número dos canais inicial e final.
        F7 = (2u bytes) STARTCHAN = Start Channel number
        F8 = (2u bytes) STOPCHAN = Stop Channel number
        """
        return bin2dec(self.block_data[24:26], False), bin2dec(self.block_data[26:28], False)

    def _get_num_meas(self) -> int:
        """
        Base method for NAMAL in this module.
        Has to be overriden in chield class.
        """
        pass

    def _get_antenna(self) -> int:
        """
        Base method for ANTUID in this module.
        Has to be overriden in chield class.
        """
        pass

    def _get_processing(self) -> Tuple[int, str]:
        """
        Base method for PROC in this module.
        Has to be overriden in chield class.
        Use instance variable 'dict_processing' declared in this class body.

        0 = single measurement,
        1 = average,
        2 = peak,
        3 = minimum
        """
        pass

    def _get_unit(self) -> Union[str, int]:
        """
        Base method for DTYPE in this module.
        Has to be overriden in chield class.
        Use instance variable 'dict_unit' declared in this class body.

        DTYPE = Data Type
        0 = % (não documentado)
        1 = dBm
        2 = dBuV/m
        """
        pass

    def _get_global_error_code(self) -> int:
        """
        Base method for GERROR in this module.
        Has to be overriden in chield class.
        """
        pass

    def _get_global_flags_code(self) -> int:
        """
        Base method for GFLAGS in this module.
        Has to be overriden in chield class.
        """
        pass

    def _get_group_id(self) -> int:
        """
        Base method for GROUPID in this module.
        Has to be overriden in chield class.
        """
        pass

    def _get_len_padding(self) -> int:
        """
        Base method for NPAD in this module.
        Has to be overriden in chield class.
        """
        pass

    def _get_number_data_points(self) -> int:
        """
        Base method for NPAD in this module.
        Has to be overriden in chield class.
        """
        return 0

    def point_freq_mega(self, index: int) -> float:
        """
        self, int > float
        :param index: Número da posição do canal (ou 'step') desejado.
        :return: O valor em MHz do início do canal (ou 'step').

        index é a posição de certo valor em "DataType65._get_data()"
        """
        #bw = self._get_stop_mega() - self._get_start_mega()
        #passo = self.bw/(self._get_number_data_points()-1)
        return self._get_start_mega()+(index*self.passo)


    def _list_freq_mega(self) -> List[float]:
        """
        self > float, float, ...
        :return: Lista com cada frequencia a que pertence o valor de 'TimedBlock._get_data()'.

        Útil para criar cabeçalhos de tabelas ou gráficos.
        """
        return [self.point_freq_mega(index) for index in range(self._get_number_data_points())]

    # def __len__(self):
    #     """
    #
    #     :return: Uma descrição quantitativa do bloco.
    #     """
    #     pass

    def __str__(self) -> str:
        """
        self > str
        :return: Uma descrição qualitativa do bloco.
        """
        """

        :return:
        """
        if self._get_processing()[0] > 1:
            return "Bloco cronometrado do tipo %s com %s amostras agrupadas em %s." \
               % (self._get_block_type(), self._get_num_meas(), self._get_processing()[1])
        else:
            return "Bloco cronometrado do tipo %s com uma única amostra" % self._get_block_type()

    datetime_tuples: Tuple[Tuple[int, int, int, int], Tuple[int, int, int, int], Tuple[int]] = \
        property(fget=_get_datetime_tuples)
    datetime_stamp: datetime = property(fget=_get_datetime_stamp)
    start_mega: int = property(fget=_get_start_mega)
    stop_mega: int = property(fget=_get_stop_mega)
    freq_band_mega: Tuple[int, int] = property(fget=_get_freq_band_mega)
    freq_band_mili: Tuple[int, int] = property(fget=_get_freq_band_mili)
    channel_interval: Tuple[int, int] = property(fget=_get_channel)
    # As classes herdeiras também devem possuir as seguintes propriedades:
    # num_meas = property(fget=_get_num_meas)
    # ant_id = property(fget=_get_antenna)
    # processing = property(fget=_get_processing)
    # unit = property(fget=_get_unit)
    # global_error_code = property(fget=_get_global_error_code)
    # global_flags_code = property(fget=_get_global_flags_code)
    # group_id = property(fget=_get_group_id)
    data_points = property(fget=_get_number_data_points)
    list_freq_mega = property(fget=_list_freq_mega)

### Data Type 21 – Unit and Job Information

In [194]:
#export
class DataType21(Block):
    """
    Data Type 21 – Unit and Job Information
    Tipo de bloco existente a partir da versão 3 do script LOGGER.
    F0 = (16 bytes) HOSTNAME = Unit Hostname.
                               Null terminated string (up to length 15), padded with nulls.
    F1 = (4 bytes) TEXT1LEN = Free Text length.
                              Including null termination and padding (must be a whole number of 4 bytes).
    F2 = (TEXT1LEN) TEXT1 = Null terminated Free Text – unit_info.
                            From ‘unit_info’ in cfg file Padded with \0 bytes,
                            E.g. could include
                                ‘method=blah;
                                other_key=blerg’ etc
                            keyword lookup in later analysis.
    F3 = (4 bytes) TEXT2LEN = Free Text length.
                              Including null termination and padding (must be a whole number of 4 bytes).
    F4 = (TEXT4LEN) TEXT4 = Null terminated Free Text – method.
                            From ‘method’ in cfg file.
    """

    # def __init__(self) -> None:
    #     # Block.__init__(self)
    #     self.text1_len = self._get_text1_len()
    #     self.text2_len = self._get_text2_len()

    def _get_hostname(self) -> str:
        """
        Retorna o campo HOSTNAME que contém o "Unit Hostname".
        :return:
        """
        return bin2str(self.block_data[:16])

    def _get_text1_len(self) -> int:
        """
        Retorna o tamanho do campo TEXT1 que contém o ‘unit_info’ no arquivo cfg.
        :return:
        """
        return bin2dec(self.block_data[16:20])

    def _get_unit_info(self) -> str:
        """
        Retorna o campo TEXT1 que contém o ‘unit_info’ no arquivo cfg.
        :return:
        """
        final = 20 + self._get_text1_len()
        return bin2str(self.block_data[16:final])

    def _get_text2_len(self) -> int:
        """
        Retorna o tamanho do campo TEXT2 que contém o ‘unit_info’ no arquivo cfg.
        :return:
        """
        inicio = 20 + self._get_text1_len()
        final = 4 + inicio
        return bin2dec(self.block_data[inicio:final])

    def _get_method(self) -> str:
        """
        Retorna o campo TEXT2 que contém o ‘method’ no arquivo cfg.
        :return:
        """
        inicio = 24 + self._get_text1_len()
        final = inicio + self._get_text2_len()
        return bin2str(self.block_data[inicio:])

    hostname: str = property(fget=_get_hostname)
    unit_info: str = property(fget=_get_unit_info)
    method: str = property(fget=_get_method)

### Data Type 22 – Data Thread Information

In [195]:
class DataType22(Block):
    """
    Data Type 22 – Data Thread Information
    Tipo de bloco existente somente na versão 3 do script LOGGER:
        - Subtitui o Data Type 3.
        - subtituído por Data Type 24 a partir da versão 4.
    F0 = (4 bytes) TEXTLEN = Free Text length.
                              Including null termination and padding (must be a whole number of 4 bytes).
    F1 = (TEXTLEN) TEXT = Null terminated Free Text description of the measurement thread - desc.
                          Padded with \0 bytes
    """
    # todo: implementar somente se necessário futuramente
    pass

### Data Type 24 – Data Thread Information

In [196]:
#export
class DataType24(Block):
    """
    Data Type 24 – Data Thread Information.
    Tipo de bloco existente a partir da versão 4 do script LOGGER.

    F0 = (4 bytes) GROUPID = ID used to group sets of data
                             0 = not a member of a group
    F1 = (4 bytes) TEXTLEN = Free Text Length
    F2 = (TEXTLEN) TEXT = Null terminated Free Text
                    description of the measurement thread - desc
    """
    def _get_group_id(self) -> int:
        """
        self > int
        :return: Número identificador do grupo. Zero significa que não faz parte de nenhum grupo.

        F0 = (4 bytes) GROUPID = ID used to group sets of data
                             0 = not a member of a group
        """
        return bin2dec(self.block_data[:4])

    def _get_len_text(self) -> int:
        """
        self > int
        :return: Comprimento do texto incluindo bytes nulos ao final (número inteiro de 4 bytes).

        F1 = (4 bytes) TEXTLEN = Free Text Length
        """
        return bin2dec(self.block_data[4:8])

    def _get_text(self) -> str:
        """
        self > str
        :return: Texto armazenado no bloco sem os bytes nulos ao final.

        F2 = (TEXTLEN) TEXT = Null terminated Free Text
                    description of the measurement thread - desc
        """
        return bin2str(self.block_data[8:])

    group_id: int = property(fget=_get_group_id)
    len_text: int = property(fget=_get_len_text)
    text: str = property(fget=_get_text)

### Data Type 40 – GPS Data

In [197]:
#export
class DataType40(Block):
    """
    O Bloco do tipo 40 carrega dados de GPS.

    Data Type40 - GPS Data
    """

    def _get_wallclock_date_stamp(self) -> Tuple[int, int, int, int]:
        """
        self > int, int, int, int
        :return: dia, mês, ano, reserva

        F0 = (4 bytes) WALLDATE = Wall Clock Start Date of measurements
        """
        return bin2date(self.block_data[:4])

    def _get_wallclock_time_stamp(self) -> Tuple[int, int, int, int]:
        """
        self > int, int, int, int
        :return: horas, minutos, segundos, décimos de segundo

        F1 = (4 bytes) WALLTIME = Wall Clock Start Time
        """
        return bin2time(self.block_data[4:8])

    def _get_wallclock_time_stamp_nanosecs(self) -> int:
        """
        self > int
        :return: nanosegundos

        F2 = (4u bytes) WALLNANO = Wall Clock Start Time Nanoseconds
        """
        return bin2dec(self.block_data[8:12], False)

    def _get_wallclock_date_time_stamp(self) -> Tuple[Tuple[int, int, int, int], Tuple[int, int, int, int], int]:
        """
        self > int, int, int, int, int, int, int, int
        :return: dia, mês, ano, reserva, horas, minutos, segundos, décimos de segundo
        """
        return \
            self._get_wallclock_date_stamp(),\
            self._get_wallclock_time_stamp(),\
            self._get_wallclock_time_stamp_nanosecs()

    def _get_gps_date_stamp(self) -> Tuple[int, int, int, int]:
        """
        self > int, int, int, int
        :return: dia, mês, ano, reserva

        F3 = (4 bytes) WALLDATE = Wall Clock Start Date of measurements
        """
        return bin2date(self.block_data[12:16])

    def _get_gps_time_stamp(self) -> Tuple[int, int, int, int]:
        """
        self > int, int, int, int
        :return: horas, minutos, segundos, décimos de segundo

        F4 = (4 bytes) WALLTIME = Wall Clock Start Time
        """
        return bin2time(self.block_data[16:20])

    def _get_gps_date_time_stamp(self)-> Tuple[Tuple[int, int, int, int], Tuple[int, int, int, int], int]:
        """
        self > int, int, int, int, int, int, int, int
        :return: dia, mês, ano, reserva, horas, minutos, segundos, décimos de segundo
        """
        return self._get_gps_date_stamp(), self._get_gps_time_stamp()

    def _get_posicional_status(self) -> Union[str, int]:
        """
        self > int
        :return: "status" do GPS conforme documentação.

        F5 = (1 byte) Posicional Fix and Status.

        if status = 1:
            0 = No Fix
            1 = Standard GPS
            2 = Differencial GPS.
        if status = 0:
            Set to zero.
        """
        status_dict = {0:'No Fix', 1:'Standard GPS', 2:'Differential GPS'}
        status = bin2dec(self.block_data[20:21])
        return status_dict.get(status, status)

    def _get_view_sats(self) -> int:
        """
        self > int
        :return: Número de satélites em visada.

        F6 = (1 byte) Sattelites in view

        0 = bad, 1+ better
        """
        return bin2dec(self.block_data[21:22])

    def _get_heading(self) -> float:
        """
        self > float
        :return: Azimute de direção.

        F7 = (2 bytes) Heading = Degrees * 100
        """
        return bin2dec(self.block_data[22:24])/100

    def _get_latitude(self) -> float:
        """
        self > float
        :return: Latitude em graus decimais. Valores positivos para emisferio Norte e negativos para Sul.

        F8 = (4 bytes) Latitude = Degrees * 1000000: +ve=N, -ve=S
        """
        return bin2dec(self.block_data[24:28]) / 1000000

    def _get_longitude(self) -> float:
        """
        self > float
        :return: Longitude em graus decimais. Valores positivos para Leste e negativos para Oeste de greenwich.

        F9 = (4 bytes) Longitude = Degrees * 1000000: +ve=E, -ve=W
        """
        return bin2dec(self.block_data[28:32]) / 1000000

    def _get_speed(self) -> float:
        """
        self > float
        :return: Velocidade em quilometros por hora.

        F10 = (4 bytes) Speed = kph * 1000
        """
        return bin2dec(self.block_data[32:36]) / 1000

    def _get_altitude(self) -> float:
        """
        :return: Altidude do nivel do mar em metros.

        self > float

        F11 = (4 bytes) Altitude = Metres * 1000
        """
        return bin2dec(self.block_data[36:40]) / 1000

    wallclock_date_time_stamp = property(fget=_get_wallclock_date_time_stamp)
    gps_date_time_stamp = property(fget=_get_gps_date_time_stamp)
    gps_status = property(fget=_get_posicional_status)
    num_sattelites = property(fget=_get_view_sats)
    heading = property(fget=_get_heading)
    latitude = property(fget=_get_latitude)
    longitude = property(fget=_get_longitude)
    speed = property(fget=_get_speed)
    altitude = property(fget=_get_altitude)

### Data Type 41 – Timed Free Text Information

In [198]:
#export
class DataType41(Block):
    """
    O Bloco do tipo 41 é uma informação em texto livre com marca temporal
    O tamanho total em bytes é (48 + NTEXT)

    F0 = (4 bytes) Date from Wall Clock (dd/mm/yy/null)
    F1 = (4 bytes) Time from Wall Clock (hh/mm/ss/cc)
    F2 = (4 bytes - 32 bit unsigned integer) Wall Clock Start Time Nanoseconds. Must be 0 if cc>0.
    F3 = (32 bytes) Fixed Length Data Type Identifier, null terminated string
        Tells the software how to deal with the free text.
        Defined types:
        - LOGGER_NAME: Application name
        - LOGGER_VERSION: Application version
        - AUDIT: Audit process output
        - GPRS: GPRS message
        - GSM: Cell survey information
        - INFO: Voltages, currents and temperatures
        - LED: LED status
        - MASK: Mask status
        - MESSAGE: Message text
        - NMEA: GSM NMEA text
        - SNMP: SNMP Message text
        - CONF: NCPD Configuration
    F4 = (4 bytes) NTEXT = Free Text Length Including null termination and padding (must be a whole number of 4 bytes)
    F5 = (NTEXT bytes) Null terminated Free Text
    """
    def _get_date_stamp(self) -> Tuple[int, int, int, int]:
        """
        self> int,int,int,int
        :return: dia, mês, ano, reserva

        F0 = (4 bytes) Date from Wall Clock (dd/mm/yy/null)
        """
        return bin2date(self.block_data[:4])

    def _get_time_stamp(self) -> Tuple[int, int, int, int]:
        """
        self> int,int,int,int
        :return: horas, minutos, segundos, décimos de segundo

        F1 = (4 bytes) Time from Wall Clock (hh/mm/ss/cc)
        """
        return bin2time(self.block_data[4:8])

    def _get_time_stamp_nanosecs(self) -> int:
        """
        self> int
        :return: nanosegundos

        F2 = (4 bytes - 32 bit unsigned integer) Wall Clock Start Time Nanoseconds.
                    Must be 0 if cc>0.
        """
        return bin2dec(self.block_data[8:12], False),

    def _get_date_time_stamp(self) -> Tuple[Tuple[int, int, int, int], Tuple[int, int, int, int], int]:
        """
        self > int, int, int, int, int, int, int, int
        :return: dia, mês, ano, reserva, horas, minutos, segundos, décimos de segundo
        """
        return self._get_date_stamp(), self._get_time_stamp(), self._get_time_stamp_nanosecs()

    def _get_identifier(self) -> str:
        """
        self > str
        :return: Identificador Descritivo (texto).

        F3 = (32 bytes) Fixed Length Data Type Identifier, null terminated string
        Tells the software how to deal with the free text.
        Defined types:
        - LOGGER_NAME: Application name
        - LOGGER_VERSION: Application version
        - AUDIT: Audit process output
        - GPRS: GPRS message
        - GSM: Cell survey information
        - INFO: Voltages, currents and temperatures
        - LED: LED status
        - MASK: Mask status
        - MESSAGE: Message text
        - NMEA: GSM NMEA text
        - SNMP: SNMP Message text
        - CONF: NCPD Configuration
        """
        return bin2str(self.block_data[12:44])

    def _get_len_text(self) -> int:
        """
        self > int
        :return: Número de caracteres do texto.

        F4 = (4 bytes) NTEXT = Free Text Length
                    Including null termination and padding
                    (must be a whole number of 4 bytes)
        """
        return bin2dec(self.block_data[44:48])

    def _get_text(self) -> str:
        """
        self > str
        :return: O texto armazenado no bloco.
        F5 = (NTEXT bytes) Null terminated Free Text
        """
        return bin2str(self.block_data[48:])

    date_time_stamp = property(fget=_get_date_time_stamp)
    identifier = property(fget=_get_identifier)
    len_text = property(fget=_get_len_text)
    text = property(fget=_get_text)

### Data Type 63 – Spectral Data

In [199]:
#export
class DataType63(TimedBlock):
    """
    O Bloco do tipo 65 carrega dados de Nível por Canal de Frequência.

    Data Type 63 – Spectral Data

    F0 a F8, F10 a F13, F15 a F17 e F20 a F21 já inicializados na classe TimedBlock.

   *F0 = (4 bytes) WALLDATE = Wall Clock Start Date of measurements
   *F1 = (4 bytes) WALLTIME = Wall Clock Start Time
   *F2 = (4u bytes) WALLNANO = Wall Clock Start Time Nanoseconds
   *F3 = (2u bytes) STARTMEGA = Start Frequency MHz
   *F4 = (4 bytes) STARTMILLI = Start Frequency mHz
   *F5 = (2u bytes) STOPMEGA = Stop Frequency MHz
   *F6 = (4 bytes) STOPMILLI = Stop Frequency mHz
   *F7 = (2u bytes) STARTCHAN = Start Channel number
   *F8 = (2u bytes) STOPCHAN = Stop Channel number
    F9 = (4 bytes) SAMPLE = Duration of sampling.
                    Time taken by the FPGA and Radio to execute command in µs
    F10 =(4 bytes) NAMAL = Amalgamated  Results
    F11 = (1u byte) ANTUID Antenna number [ 0- 255]
    F12 = (1 byte) PROC = Processing
                    0 = single measurement,
                    1 = average,
                    2 = peak,
                    3 = minimum
    F13 = (1 byte) DTYPE = Data Type
                    1 = dBm,
                    2 = dBuV/m
    F14 = (1 byte) OFFSET = Data level offset in DTYPE units
                    2’s Complement, range [-128, 127].
    F15 = (1 byte)
    F16 = (1 byte)
    F17 = (1 byte)
    F18 = (2 bytes)
    F19 = (2 bytes)
    F20 = (1 byte)
    F21 = (4 bytes)
    F22 = (4*NTUN bytes)
    F23 = (NAGC bytes)
    F24 = (NDATA bytes)
    F25 = (NPAD bytes)
    """
    
    def __init__(self, data)-> None:
        super().__init__(data)
   
    def _get_sample_spent_time_microsecs(self) -> int:
        """
        self > int
        :return: O tempo levado pelo radio e FPGA para a aquisição da amostra, em microsegundos.

        F9 = (4 bytes) SAMPLE = Duration of sampling.
                    Time taken by the FPGA and Radio to execute command in µs
        """
        return bin2dec(self.block_data[28:32])

    def _get_num_meas(self) -> int:
        """
        self > int
        :return: O número de resultados agrupados. Se for 1 equivale a uma única medida.

        Overrides method in TimedBlock.

        F10 = (4 bytes) NAMAL = Amalgamated Results
                    i.e. ‘number of loops’. Equal to 1 if single measurement.
        """
        return bin2dec(self.block_data[32:36])

    def _get_antenna(self) -> int:
        """
        self > int
        :return: o ID da antena usada na medida.

        Overrides method in TimedBlock.

        F11 = (1u bytes) ANTUID Antenna number [ 0- 255]
        """
        return bin2dec(self.block_data[32:33], False)

    def _get_processing(self) -> Union[str, int]:
        """
        self > (int, str)
        :return: O código e a descrição do tipo de processamento aplicado à medida.

        Overrides method in TimedBlock.

        F12 = (1 byte) PROC = Processing
                    0 = single measurement,
                    1 = average,
                    2 = peak,
                    3 = minimum
        """
        proc = bin2dec(self.block_data[37:38])
        return TimedBlock.dict_processing.get(proc, proc)

    def _get_unit(self) -> Union[str, int]:
        """
        self > str or int
        :return: A unidade de medida.
                Retorna um número inteiro se a unidade não foi documentada.

        Overrides method in TimedBlock.

        todo: Não está funcionando, todos os dados retornam o valor 0!

        F13 = DTYPE = Data Type
            0 = % (não documentado)
            1 = dBm
            2 = dBuV/m
        """
        unit: int = bin2dec(self.block_data[38:39])
        return TimedBlock.dict_unit.get(unit, unit)

    def _get_level_offset(self) -> int:
        """
        self > int
        :return: O deslocamento em nível adotado na compactação dos dados binários.
                 A unidade é dada por self._get_unit().

        F14 = (1 bytes) OFFSET = Data level offset in DTYPE units
                                2’s Complement, range [-128, 127].
        """
        return bin2dec(self.block_data[39:40])

    def _get_global_error_code(self) -> int:
        """
        self > int
        :return: O código de erro global.

        Overrides method in TimedBlock.

        F15 = (1 byte) GERROR = Global Error Code.
                    Radio or processing global error code

        The radio error codes and flags can be provided on request.
        """
        return bin2dec(self.block_data[40:41])

    def _get_global_flags_code(self) -> int:
        """
        self > int
        :return: Códigos de alertas globais ou de processamento do radio.

        Overrides method in TimedBlock.

        F16 = (1 byte) GFLAGS = Global clipping flags etc.
                    Radio or processing global flags.

        The radio error codes and flags can be provided on request.
        """
        return bin2dec(self.block_data[41:42])

    def _get_group_id(self) -> int:
        """
        self > int
        :return: O ID do grupo à qual a medida pertence.
                 Caso 0 não pertence a nenhum grupo.
                 Use a classe 'DataType24' para detalhes do grupo.

        Overrides method in TimedBlock.

        F17 = (1 bytes) GROUPID = ID used to group sets of data
                    0 = not a member of a group
        """
        return bin2dec(self.block_data[42:43])

    def _get_len_tunning_info(self) -> int:
        """
        self > int
        :return: 0 ou igual à quantidade de valores de AGC usados na amostra.

        F18 = (2 bytes) NTUN= Number of 4 byte Tuning info blocks. 0 or NAGC
        """
        return bin2dec(self.block_data[43:45])

    def _get_len_agc(self) -> int:
        """
        self > int
        :return: 0 ou igual à quantidade de valores de "tunings" usados na amostra.

        F19 = (2 bytes) NAGC = Number of single byte AGC values
                                Can be 0, or = number of tunings in the sweep
        """
        return bin2dec(self.block_data[45:47])

    def _get_len_padding(self) -> int:
        """
        self > int
        :return: Valor que varia de 0 a 3 indicando o preenchimento nulo para manter o tamanho do bloco (em bytes) fixo.

        Overrides method in TimedBlock.

        A extração do NPAD (quantidade) tem pouca utilidade prática.
        A extração do Padding (valor) não tem utilidade prática.

        F20 = (1 byte) NPAD = Number of bytes of padding. 0-3
        (NPAD bytes) Padding = As \0 bytes
        """
        return bin2dec(self.block_data[39:40])

    def _get_number_data_points(self) -> int:
        """
        self > int
        :return: O número de canais (ou "steps") que dividem igualmente a largura de banda .

        Overrides method in TimedBlock.

        F21 = (4 bytes) NDATA = Number of single byte data points.
                        Number of equal width channels dividing the reported frequency width
        """
        return bin2dec(self.block_data[48:52])

    def _get_rbw(self) -> float:
        """
        Retorna o RBW calculado a partir de STARTMEGA, STOPMEGA e NDATA.
        :return: RBW (Hz)
        """
        return int(((self.stop_mega - self.start_mega) / (self.data_points - 1) * 1000000))

    def _get_tunning_info_array(self) -> bytes:
        """
        self > bytes
        :return: Binario bruto de informações do 'tunning'.

        F22 = (4*NTUN bytes) = Array of Tuning info blocks
                        One block per tuning (1 or 10 MHz). See section 11.7.

        todo: implementar um método para traduzir o 'Tuning info block' de acordo com a seção 11.7 da documentação.
        """
        start = 52  # inicia aqui
        stop = start+(self._get_len_tunning_info()*4)  # termina aqui
        return self.block_data[start:stop]

    def _get_agc_array(self) -> bytes:
        """
        self > bytes
        :return: Binario bruto com toda a matriz de AGC.

        F23 = (NAGC  bytes) = Array of AGC values As dB in single unsigned byte.
                        Current actual values are 0..63.

        todo: implementar um método que forneça a matriz de números inteiros do AGC.
        """
        start = 52+(self._get_len_tunning_info()*4)  # inicia após F22
        stop = start+(self._get_len_agc())  # termina aqui
        return self.block_data[start:stop]

    def _get_data(self) -> List[float]:
        """
        self > List[float]
        :return: A lista de todas as medidas de espectro do bloco em 'dB'.

        F24 = (NDATA  bytes) = Array of dB spectrum data points.
        Data points in a byte representing  the power in dBm in 0.5 dBm steps
        """
        start = 52+(self._get_len_tunning_info()*4)+(self._get_len_agc())  # inicia após F23
        stop = start+(self._get_number_data_points())  # termina aqui
        offset = self._get_level_offset()
        return [decode_spectrum(d,offset) for d in self.block_data[start:stop]]

    spent_time_microsecs: int = property(fget=_get_sample_spent_time_microsecs)
    level_offset: int = property(fget=_get_level_offset)
    # len_tunning_info: int = property(fget=_get_len_tunning_info)
    tunning_info_array: bytes = property(fget=_get_tunning_info_array)
    # len_agc: int = property(fget=_get_len_agc)
    agc_array: bytes = property(fget=_get_agc_array)
    block_data: List[float] = property(fget=_get_data)
    num_meas: int = property(fget=_get_num_meas)
    ant_id: int = property(fget=_get_antenna)
    processing: Union[str, int] = property(fget=_get_processing)
    unit: Union[str, int] = property(fget=_get_unit)
    global_error_code: int = property(fget=_get_global_error_code)
    global_flags_code: int = property(fget=_get_global_flags_code)
    group_id: int = property(fget=_get_group_id)
    data_points: int = property(fget=_get_number_data_points)
    rbw: float = property(fget=_get_rbw)

### Data Type 65 – Frequency Channel Occupancy (ITU-R SM.1880)

In [200]:
#export
class DataType65(TimedBlock):
    """
    O Bloco do tipo 65 carrega dados de Taxa de Ocupação Espectral por Canal de Frequência (ITU-R SM.1880).

    O tamanho total em bytes é (48 + NDATA + NPAD)

    F0 a F16 e F19 já inicializados na classe TimedBlock.

   *F0 = (4 bytes) WALLDATE = Wall Clock Start Date of measurements
   *F1 = (4 bytes) WALLTIME = Wall Clock Start Time
   *F2 = (4u bytes) WALLNANO = Wall Clock Start Time Nanoseconds
   *F3 = (2u bytes) STARTMEGA = Start Frequency MHz
   *F4 = (4 bytes) STARTMILLI = Start Frequency mHz
   *F5 = (2u bytes) STOPMEGA = Stop Frequency MHz
   *F6 = (4 bytes) STOPMILLI = Stop Frequency mHz
   *F7 = (2u bytes) STARTCHAN = Start Channel number
   *F8 = (2u bytes) STOPCHAN = Stop Channel number
    F9 = (4 bytes) NAMAL = Amalgamated Results
                    i.e. ‘number of loops’. Equal to 1 if single measurement.
    F10 = (1u bytes) ANTUID Antenna number [ 0- 255]
    F11 = (1 byte) PROC = Processing
                    0 = single measurement,
                    1 = average,
                    2 = peak,
                    3 = minimum
    F12 = (2 bytes) DTYPE = Data Type
                    1 = dBm,
                    2 = dBuV/m
    F13 = (1 byte) GERROR = Global Error Code. Radio or processing global error code
    F14 = (1 byte) GFLAGS = Global clipping flags etc. Radio or processing global flags:
    F15 = (1 bytes) GROUPID = ID used to group sets of data
                    0 = not a member of a group
    F16 = (1 byte) NPAD = Number of bytes of padding. 0-3
    F17 = (2 bytes) THRESH = Threshold Level in DTYPE
    F18 = (2 bytes) DURATION = Duration of sampling. In seconds  (i.e. 300, 900, 1800, 2600)
    F19 = (4 bytes) NDATA = Number of single byte data points.
                    Number of equal width channels dividing the reported frequency width
    F20 = (NDATAu bytes) Array of data points.
                    Each data point is stored as a single byte number
                    representing  the percentage (0..100 % in 0.5 steps)
    F21 = (NPAD bytes) Padding. As \0 bytes
    """

    def _get_num_meas(self) -> int:
        """
        self > int
        :return: O número de resultados agrupados. Se for 1 equivale a uma única medida.

        Overrides method in TimedBlock.

        F9 = (4 bytes) NAMAL = Amalgamated Results
                    i.e. ‘number of loops’. Equal to 1 if single measurement.
        """
        return bin2dec(self.block_data[28:32])

    def _get_antenna(self) -> int:
        """
        self > int
        :return: o ID da antena usada na medida.

        Overrides method in TimedBlock.

        F10 = (1u bytes) ANTUID Antenna number [ 0- 255]
        """
        return bin2dec(self.block_data[32:33], False)

    def _get_processing(self) -> Union[str, int]:
        """
        self > (int, str)
        :return: O código e a descrição do tipo de processamento aplicado à medida.

        Overrides method in TimedBlock.

        F11 = (1 byte) PROC = Processing
                    0 = single measurement,
                    1 = average,
                    2 = peak,
                    3 = minimum

        Data Type 65, Field 11
        Data Type 63, Field 12
        """
        proc = bin2dec(self.block_data[33:34])
        return TimedBlock.dict_processing.get(proc, proc)

    def _get_unit(self) -> Union[str, int]:
        """
        self > str or int
        :return: A unidade de medida.
                Retorna um número inteiro se a unidade não foi documentada.

        Overrides method in TimedBlock.

        todo: Não está funcionando, todos os dados retornam o valor 0!

        F12 = (1 byte) DTYPE = Data Type
            0 = % (não documentado)
            1 = dBm
            2 = dBuV/m
        """
        unit = bin2dec(self.block_data[34:36])
        return TimedBlock.dict_unit.get(unit, unit)

    def _get_global_error_code(self) -> int:
        """
        self > int
        :return: O código de erro global.

        Overrides method in TimedBlock.

        F13 = (1 byte) GERROR = Global Error Code.
                    Radio or processing global error code

        The radio error codes and flags can be provided on request.
        """
        return bin2dec(self.block_data[36:37])

    def _get_global_flags_code(self) -> int:
        """
        self > int
        :return: Códigos de alertas globais ou de processamento do radio.

        Overrides method in TimedBlock.

        F14 = (1 byte) GFLAGS = Global clipping flags etc.
                    Radio or processing global flags.

        The radio error codes and flags can be provided on request.
        """
        return bin2dec(self.block_data[37:38])

    def _get_group_id(self) -> int:
        """
        self > int
        :return: O ID do grupo à qual a medida pertence.
                 Caso 0 não pertence a nenhum grupo.
                 Use a classe 'DataType24' para detalhes do grupo.

        Overrides method in TimedBlock.

        F15 = (1 bytes) GROUPID = ID used to group sets of data
                    0 = not a member of a group
        """
        return bin2dec(self.block_data[38:39])

    def _get_len_padding(self) -> int:
        """
        self > int
        :return: Valor que varia de 0 a 3 indicando o preenchimento nulo para manter o tamanho do bloco (em bytes) fixo.

        Overrides method in TimedBlock.

        A extração do NPAD (quantidade) tem pouca utilidade prática.
        A extração do Padding (valor) não tem utilidade prática.

        F16 = (1 byte) NPAD = Number of bytes of padding. 0-3
        (NPAD bytes) Padding = As \0 bytes
        """
        return bin2dec(self.block_data[47:48])

    def _get_threshold(self) -> int:
        """
        self > int
        :return: O valor do limiar de detecção. A unidade é dada por self._get_unit().

        F17 = (2 bytes) THRESH = Threshold Level in DTYPE
        """
        return bin2dec(self.block_data[40:42])

    def _get_duration_seconds(self) -> int:
        """
        self > int
        :return: A duração da amostra em segundos.

        F18 = (2 bytes) DURATION = Duration of sampling. In seconds
                    (i.e. 300, 900, 1800, 2600)
        """
        return bin2dec(self.block_data[42:44])

    def _get_number_data_points(self) -> int:
        """
        self > int
        :return: O número de canais (ou "steps") que dividem igualmente a largura de banda .

        Overrides method in TimedBlock.

        F19 = (4 bytes) NDATA = Number of single byte data points.
                        Number of equal width channels dividing the reported frequency width
        """
        return bin2dec(self.block_data[44:48])

    def _get_data(self) -> List[float]:
        """
        self > -> List[float]
        :return: A lista de todas as medidas de ocupação do bloco em '%'.

        F20 = (NDATAu bytes) Array of data points.
                    Each data point is stored as a single byte number
                    representing  the percentage (0..100 % in 0.5 steps)
        """
        data_array = []
        pos_inicial = 48
        for delta_pos in range(self._get_number_data_points()):
            byte_index = pos_inicial + delta_pos
            # cada "data point" vai de 0 a 200 em passos de 1
            # que equivale a 0% a 100% em passos de 0,5%
            # por isso o valor é dividido por 2
            data_array.append(bin2dec(self.block_data[byte_index:byte_index+1], False)/2)
        return data_array

    threshold: int = property(fget=_get_threshold)
    duration_seconds: int = property(fget=_get_duration_seconds)
    block_data: List[float] = property(fget=_get_data)
    num_meas: int = property(fget=_get_num_meas)
    ant_id: int = property(fget=_get_antenna)
    processing: Union[str, int] = property(fget=_get_processing)
    unit: Union[str, int] = property(fget=_get_unit)
    global_error_code: int = property(fget=_get_global_error_code)
    global_flags_code: int = property(fget=_get_global_flags_code)
    group_id: int = property(fget=_get_group_id)
    data_points: int = property(fget=_get_number_data_points)


class Error(Exception):
    """Base class for exceptions in this module."""
    pass


class InputFileError(Error):
    """
    Exception raised for errors in the input.

    Attributes:
        filename -- input file in which the error occurred
        message -- explanation of the error
    """

    def __init__(self, message: str) -> None:
        self.message = message

def retorna_classe(type_num: int) -> Any:
    map_type_class = {
        24: DataType24,
        40: DataType40,
        41: DataType41,
        63: DataType63,
        65: DataType65
    }
    return map_type_class[type_num]

In [201]:
from nbdev.export import notebook2script
notebook2script()

Converted 00_main.ipynb.
Converted 01_parser.ipynb.
Converted 02_utils.ipynb.
Converted 03_test_run.ipynb.
Converted 04_blocks.ipynb.
Converted index.ipynb.
